# Adapter eval vs predict
TThis is to verify that eval and predict give similar results when using adapters.

It is beacause the adapters use in-built image processing and data loading but what we are interesetd is if they will handle images during the real inference the same way.

Values may differ slightly due to different thresholds used in eval and predict, but should be acceptably close - 0.05 map_50_90 away (arbitraty number concluded after various tests and observations).


In [1]:
from ml_carbucks import DATA_DIR
from ml_carbucks.adapters.EfficientDetAdapter import EfficientDetAdapter
from ml_carbucks.adapters.FasterRcnnAdapter import FasterRcnnAdapter
from ml_carbucks.adapters.UltralyticsAdapter import RtdetrUltralyticsAdapter, YoloUltralyticsAdapter
from ml_carbucks.ensemble.EnsembleModel import EnsembleModel
from ml_carbucks.utils.logger import setup_logger

logger = setup_logger("adapter_eval_vs_predict")

ensemble = EnsembleModel(
    classes=["scratch", "dent", "crack"],
    adapters=[
        YoloUltralyticsAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 32,
                "epochs": 27,
                "lr": 0.0015465639515144544,
                "momentum": 0.3628781599889685,
                "weight_decay": 0.0013127041660177367,
                "optimizer": "NAdam",
                "verbose": False,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_YoloUltralyticsAdaptermodel.pt",
        ),
        RtdetrUltralyticsAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 16,
                "epochs": 10,
                "lr": 0.0001141043015859849,
                "momentum": 0.424704619626319,
                "weight_decay": 0.00012292547851740234,
                "optimizer": "AdamW",
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_RtdetrUltralyticsAdaptermodel.pt",
        ),
        FasterRcnnAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 8,
                "epochs": 30,
                "lr_backbone": 2.6373762637681257e-05,
                "lr_head": 0.0011244046084737927,
                "weight_decay_backbone": 0.000796017512818448,
                "weight_decay_head": 0.0005747409908715994,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/FasterRcnnAdaptermodel.pth",
        ),
        EfficientDetAdapter(
            classes=["scratch", "dent", "crack"],
            **{
                "img_size": 384,
                "batch_size": 8,
                "epochs": 26,
                "optimizer": "momentum",
                "lr": 0.003459928723120903,
                "weight_decay": 0.0001302610542371722,
            },
            weights="/home/bachelor/ml-carbucks/results/ensemble_demos/trial_4_EfficientDetAdaptermodel.pth",
        ),
    ],
).setup()

train_datasets = [
    (
        DATA_DIR / "car_dd_testing" / "images" / "train",
        DATA_DIR / "car_dd_testing" / "instances_train_curated.json",
    )
]
val_datasets = [
    (
        DATA_DIR / "car_dd_testing" / "images" / "val",
        DATA_DIR / "car_dd_testing" / "instances_val_curated.json",
    )
]


def test_1():
    logger.info("Evaluating ensemble by evaluation from dataset")
    metrics = ensemble.evaluate_adapters_by_evaluation_from_dataset(val_datasets)  # type: ignore
    logger.info("Evaluation by evaluation results:")
    for idx, adapter in enumerate(ensemble.adapters):
        logger.info(f"Adapter: {adapter.__class__.__name__}, Metrics: {metrics[idx]}")


    return metrics


def test_2():
    logger.info("Evaluating ensemble by predict from dataset")
    metrics = ensemble.evaluate_adapters_by_predict_from_dataset(val_datasets)  # type: ignore
    logger.info("Evaluation by predict results:")
    for idx, adapter in enumerate(ensemble.adapters):
        logger.info(f"Adapter: {adapter.__class__.__name__}, Metrics: {metrics[idx]}")
    return metrics


def test_3(m1, m2, metric_name: str):
    logger.info("Comparing results from both evaluation methods")
    for idx, adapter in enumerate(ensemble.adapters):
        logger.info(f"Adapter: {adapter.__class__.__name__}")
        logger.info(f"Metrics from evaluation: {m1[idx][metric_name]}")
        logger.info(f"Metrics from predict: {m2[idx][metric_name]}")


if __name__ == "__main__":
    m1 = test_1()
    m2 = test_2()

    test_3(m1, m2, "map_50_95")


INFO adapter_eval_vs_predict 23:39:41 | Evaluating ensemble by evaluation from dataset
INFO ml_carbucks.adapters.UltralyticsAdapter 23:39:41 | Starting evaluation...
INFO ml_carbucks.adapters.UltralyticsAdapter 23:39:41 | Converting COCO annotations to YOLO format...
INFO ml_carbucks.adapters.UltralyticsAdapter 23:39:41 | COCO to YOLO conversion completed in 0.02 seconds
INFO ml_carbucks.adapters.UltralyticsAdapter 23:39:41 | YOLO dataset YAML created at: /home/bachelor/ml-carbucks/data/car_dd_testing/instances_val_curated.yaml
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 9859.3±1392.3 MB/s, size: 789.8 KB)
val: Scanning /home/bachelor/ml-carbucks/data/car_dd_testing/labels/val.cache... 633 images, 177 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 810/810 3.1Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 51/51 14.8it/s 3.5s0.1s
                   all        810       1406       0.43      0.399      0.375      0.

100%|██████████| 102/102 [00:05<00:00, 19.76it/s]


INFO ml_carbucks.ensemble.EnsembleModel 23:40:38 | Evaluating adapter: RtdetrUltralyticsAdapter


100%|██████████| 102/102 [00:08<00:00, 12.60it/s]


INFO ml_carbucks.ensemble.EnsembleModel 23:40:46 | Evaluating adapter: FasterRcnnAdapter


100%|██████████| 102/102 [00:34<00:00,  2.92it/s]


INFO ml_carbucks.ensemble.EnsembleModel 23:41:21 | Evaluating adapter: EfficientDetAdapter


100%|██████████| 102/102 [00:13<00:00,  7.49it/s]


INFO adapter_eval_vs_predict 23:41:35 | Evaluation by predict results:
INFO adapter_eval_vs_predict 23:41:35 | Adapter: YoloUltralyticsAdapter, Metrics: {'map_50': 0.3114479184150696, 'map_50_95': 0.1649264544248581, 'classes': [1, 2, 3]}
INFO adapter_eval_vs_predict 23:41:35 | Adapter: RtdetrUltralyticsAdapter, Metrics: {'map_50': 0.44884592294692993, 'map_50_95': 0.2507704794406891, 'classes': [1, 2, 3]}
INFO adapter_eval_vs_predict 23:41:35 | Adapter: FasterRcnnAdapter, Metrics: {'map_50': 0.25654569268226624, 'map_50_95': 0.09843528270721436, 'classes': [1, 2, 3]}
INFO adapter_eval_vs_predict 23:41:35 | Adapter: EfficientDetAdapter, Metrics: {'map_50': 0.3560166358947754, 'map_50_95': 0.16611579060554504, 'classes': [1, 2, 3]}
INFO adapter_eval_vs_predict 23:41:35 | Comparing results from both evaluation methods
INFO adapter_eval_vs_predict 23:41:35 | Adapter: YoloUltralyticsAdapter
INFO adapter_eval_vs_predict 23:41:35 | Metrics from evaluation: 0.19936537404157367
INFO adapter_ev